In [ ]:
!python3 -m pip install matplotlib
!python3 -m pip install scipy
!python3 -m pip install scikit-learn

import numpy.random
from sklearn import linear_model
import matplotlib.pyplot
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from collections import Counter
import re
import numpy as np

In [ ]:
df = pd.read_csv('amazonFood 2.csv')
df = df.iloc[:1000, :]

In [ ]:
df

In [ ]:
id_to_var = {}
var_counter = 0

for product_id in df['ProductId'].unique():
    if var_counter < 26:
        var = chr(ord('a') + var_counter)
    else:
        var = chr(ord('a') + (var_counter % 26)) + str(var_counter // 26)

    id_to_var[product_id] = var
    var_counter += 1

for product_id, product_name in zip(df['ProductId'], df['productName']):
    print(f"Product Id: {product_id}, Product Name: {product_name}, Variable: {id_to_var[product_id]}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
product_var_count = {var: 0 for var in id_to_var.values()}
product_var_occurrences = {var: 0 for var in id_to_var.values()}

words = {'Negative': ['Not', 'Didn’t', 'Bad', 'Slow', 'Worst', 'Sucks', 'Sicks', 'Grossed', 'Disgust', 'Horrible'],
         'Positive': ['Good', 'Great', 'Fast', 'Amazing', 'Like', 'Delight', 'Right', 'Tasty', 'Huge', 'Big', 'Glad', 'happy']}
text_list = []
for index, row in df.iterrows():
    product_id = row['ProductId']
    text = row['Text'].lower()
    text_list.append(text)
    product_var = id_to_var[product_id]
    for word in words['Negative']:
        if word.lower() in text:
            product_var_count[product_var] -= 1
            product_var_occurrences[product_var] += 1
    for word in words['Positive']:
        if word.lower() in text:
            product_var_count[product_var] += 1
            product_var_occurrences[product_var] += 1

for product_id, product_name in zip(df['ProductId'], df['productName']):
    var = id_to_var[product_id]
    count = product_var_count[var]
    print(f"Product Id: {product_id}, Product Name: {product_name}, Variable: {var}, Count: {count}")



In [ ]:
product_var_occurrences = {var: 0 for var in id_to_var.values()}
product_names = dict(zip(df['ProductId'], df['productName']))

sum_scores = {}
for var in product_var_count:
    occurrences = product_var_occurrences[var]
    sum_scores[var] = product_var_count[var]

sum_of_scores_list = []
for var in sum_scores:
    product_id = next(key for key, value in id_to_var.items() if value == var)
    product_name = product_names[product_id]
    sum_of_scores = sum_scores[var]
    sum_of_scores_list.append(sum_of_scores)
    print(f"Variable: {var}, Product Name: {product_name}, Sum of Scores: {sum_of_scores}")

print("Sum of Scores List:", sum_of_scores_list)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model

product_names = dict(zip(df['ProductId'], df['productName']))

vectorizer = TfidfVectorizer()
text_list_vectorized = vectorizer.fit_transform(text_list)
print(text_list_vectorized)

model = linear_model.LinearRegression()
model.fit(text_list_vectorized[:198], sum_of_scores_list[:198])
new_text = "This cereal has no artificial sweeteners, is high in fiber, but has a great taste (A hard combination to find)"
new_text_vectorized = vectorizer.transform([new_text])
confidence_percentage = model.predict(new_text_vectorized)
print(f"Confidence Percentage for New Text: {confidence_percentage}")